In [1]:
import os
import pandas as pd

In [7]:
if 'data' not in os.listdir('.'):
    os.mkdir('./data')

if 'CUP' not in os.listdir('./data'):
    os.mkdir('./data/CUP')

# Input Data

Come prerequisito per il controllo dei Codici CUP bisogna inserire nella cartella `data` i seguenti elementi:

1. **Dati Opencoesione**: salvare l'estrazione dei dati OpenCoesione, in formato .zip, nella cartella 'data' [(vedi qui - link)](https://opencoesione.gov.it/it/opendata/progetti_esteso.zip)

2. **Feedback DIPE**: salvare i file di feedback forniti dal DIPE nella cartella 'data/CUP'

3. **Estrazione CUP**: salvare un'estrazione dei CUP da PAdigitale (dall'omonimo report Estrazione CUP) nella cartella 'data'

In [8]:
opencoesione_filename = '<INSERIRE_FILENAME_OPENCOESIONE>.zip'

dipe_feedback = 'CUP'

list_cup_filename = '<INSERIRE_FILENAME_ESTRAZIONE>.xlsx'

Le successive 3 celle importano i dati dalle sorgenti salvate nella cartella `data`

In [ ]:
df_OC = pd.read_csv("data/" + opencoesione_filename,
                    compression="zip",sep=";",quotechar='"',
                    low_memory=False,
                    usecols=[
                        "COD_LOCALE_PROGETTO",
                        "CUP",
                        "OC_TITOLO_PROGETTO",
                        "OC_DESCR_CICLO",
                        "OC_COD_CICLO",
                        "OC_TEMA_SINTETICO",
                        "FONDO_COMUNITARIO",
                        "OC_CODICE_PROGRAMMA",
                        "OC_DESCRIZIONE_PROGRAMMA",
                        "COD_STRUMENTO",
                        "DESCR_STRUMENTO",
                        "OC_STATO_FINANZIARIO",
                        "OC_STATO_PROGETTO",
                        "OC_STATO_PROCEDURALE",
                        ],header=0,
                        )

In [ ]:
print("\nReading data from DIPE.")


DIRNAME = "data/" + dipe_feedback
dfs = []
lenghts = []
for filename in os.listdir(DIRNAME):
    if filename.endswith(".xlsx"):
        with pd.ExcelFile(os.path.join(DIRNAME, filename)) as fl:
            data = pd.read_excel(fl, sheet_name=0, header=0).copy()
            dfs.append(data)
            lenghts.append(len(data))
df = pd.concat(dfs, axis=0, ignore_index=True).drop_duplicates()

print("\nCompleted!\n")

In [ ]:
print("\nReading data from SalesForce.")

with pd.ExcelFile(os.path.join("data", list_cup_filepath)) as fl:
    df_pad2026 = pd.read_excel(fl, sheet_name=0).copy()

print("\nCompleted!\n")

# Preparazione Dati

In questa sezione vengono effettuate alcune operazioni, molto semplici, di preparazione dei dati:
1. Formatting dei nomi delle colonne
2. Merging dei dataset
3. Eliminazione dei CUP duplicati da OpenCoesione

In [ ]:
col_list = df_pad2026.columns
df_pad2026.columns = [col.upper().replace(" ", "_") for col in col_list]

tmp_df = df_pad2026.merge(df, on="CODICE_CUP", how="left").copy()
assert len(tmp_df) == len(df_pad2026)
df_OC = df_OC.drop_duplicates(subset="CUP", keep="last").copy()

print("Checking CUP...")

## Controlli CUP

Il codice fornito consiste di una serie di operazioni eseguite sul dataframe `tmp_df`. Di seguito è una spiegazione delle diverse operazioni:

1. `DOUBLE_FUNDING_CUP`: Questa riga di codice esegue una merge tra la colonna "CUP" del dataframe `df_OC` e la colonna "CODICE_CUP" del dataframe `tmp_df`. Viene utilizzato un join interno (`how="inner"`) e viene specificata la validazione "1:m" (`validate="1:m"`). Infine, viene estratto l'array univoco dei valori della colonna "CUP" risultante dalla merge.

2. `DOUBLE_REQUEST_CUP`: Questa riga di codice esegue una serie di operazioni sul dataframe `tmp_df`. Innanzitutto, viene raggruppato il dataframe per la colonna "CODICE_CUP". Successivamente, viene applicata una funzione di filtro che seleziona solo i gruppi con una lunghezza maggiore di 1. Quindi, i risultati vengono ordinati per la colonna "CODICE_CUP" e viene estratto l'array univoco dei valori della colonna "CODICE_CUP".

3. `MISSING_CUP`: Questa riga di codice seleziona i valori univoci della colonna "CODICE_CUP" del dataframe `tmp_df` in cui la colonna "STATO_PROGETTO" è uguale a "CUP INESISTENTE".

4. `MISSING_TEMPLATE`: Questa riga di codice seleziona i valori univoci della colonna "CODICE_CUP" del dataframe `tmp_df` in cui la colonna "TEMPLATE" è mancante (NaN).

5. `map_sheet2template`: Questa riga di codice crea un dizionario che mappa i valori univoci della colonna "NOME_DECRETO" del dataframe `tmp_df` al valore moda della colonna "TEMPLATE" corrispondente.

6. `WRONG_TEMPLATE`: Questa riga di codice seleziona i valori univoci della colonna "CODICE_CUP" del dataframe `tmp_df` in cui il valore della colonna "NOME_DECRETO" dopo la sostituzione con il dizionario `map_sheet2template` non corrisponde al valore della colonna "TEMPLATE".

Queste operazioni sembrano essere finalizzate all'identificazione di determinati pattern o condizioni all'interno del dataframe `tmp_df`, come la presenza di doppie richieste di CUP, CUP mancanti, template mancanti o template errati.

In [ ]:
DOUBLE_FUNDING_CUP = pd.merge(
    df_OC["CUP"],
    tmp_df["CODICE_CUP"],
    left_on="CUP",
    right_on="CODICE_CUP",
    how="inner",
    validate="1:m",
    ).CUP.unique()

# CUP double project
DOUBLE_REQUEST_CUP = (
    tmp_df.groupby(by="CODICE_CUP")
    .filter(lambda x: len(x) > 1)
    .sort_values(by="CODICE_CUP")
    .CODICE_CUP.unique()
    )

# CUP non esistente nei sistemi DIPE
na_cup_condition = tmp_df.STATO_PROGETTO == "CUP INESISTENTE"
MISSING_CUP = tmp_df[na_cup_condition].CODICE_CUP.unique()

# Template mancante
MISSING_TEMPLATE = tmp_df[tmp_df.TEMPLATE.isna()].CODICE_CUP.unique()

# Template errato
map_sheet2template = (
    tmp_df[["NOME_DECRETO", "TEMPLATE"]]
    .groupby(["NOME_DECRETO"])
    .TEMPLATE.agg(pd.Series.mode)
    .to_dict()
    )

WRONG_TEMPLATE = []
WRONG_TEMPLATE = tmp_df[
    tmp_df.NOME_DECRETO.replace(map_sheet2template) != tmp_df.TEMPLATE
    ].CODICE_CUP.unique()

## Classificazione CUP

In [9]:
def classify_CUP(CUP: str):
    """
    Classifica il CUP in base a una scala di priorità.

    Parameters:
        CUP (str): Il codice CUP da classificare.

    Returns:
        str: La classificazione del CUP.
    """

    if CUP in DOUBLE_FUNDING_CUP:
        return "CUP già presente in OPEN COESIONE"
    elif CUP in MISSING_CUP:
        return "CUP NON PRESENTE NEL FEEDBACK DIPE"
    elif CUP in DOUBLE_REQUEST_CUP:
        return "CUP PRESENTE IN PIU' CANDIDATURE"
    elif CUP in MISSING_TEMPLATE:
        return "TEMPLATE MANCANTE"
    elif CUP in WRONG_TEMPLATE:
        return "POSSIBILE TEMPLATE ERRATO"
    else:
        return "NESSUN PROBLEMA RISCONTRATO AL MOMENTO"

# Salvataggio risultati

Il codice fornito è finalizzato eseguire ad alcune operazioni di scrittura a partire da `tmp_df`. 

Di seguito è una spiegazione dell'obiettivo e del funzionamento del codice:

1. `tmp_df["CLASSIFICAZIONE_ISSUE_CUP"] = tmp_df.CODICE_CUP.apply(classify_CUP)`: Questa riga di codice crea una nuova colonna chiamata "CLASSIFICAZIONE_ISSUE_CUP" nel dataframe `tmp_df`. La colonna viene popolata applicando una funzione chiamata `classify_CUP` alla colonna "CODICE_CUP" esistente. La funzione `classify_CUP` sembra essere definita altrove nel codice e viene utilizzata per assegnare una classificazione a ciascun valore nella colonna "CODICE_CUP".

2. `tmp_df.groupby(by=["CLASSIFICAZIONE_ISSUE_CUP"]).CODICE_CUP.count().to_csv("data/KPI_aggregati.csv", index=True)`: Questa riga di codice esegue un'operazione di raggruppamento sul dataframe `tmp_df` utilizzando la colonna "CLASSIFICAZIONE_ISSUE_CUP" come chiave di raggruppamento. Viene quindi calcolato il conteggio dei valori nella colonna "CODICE_CUP" per ciascun gruppo. Infine, i risultati vengono salvati in un file CSV chiamato "KPI_aggregati.csv" nella cartella "data".

3. `tmp_df.to_excel("data/CUP_ANALYSIS.xlsx", index=False)`: Questa riga di codice salva il dataframe `tmp_df` in un file Excel chiamato "CUP_ANALYSIS.xlsx" nella cartella "data". L'opzione `index=False` indica che l'indice del dataframe non deve essere incluso nel file Excel.

L'obiettivo del codice sembra essere quello di eseguire alcune operazioni di analisi e aggregazione sui dati nel dataframe `tmp_df`, e quindi salvare i risultati in file CSV ed Excel nella cartella `data`.

In [ ]:
tmp_df["CLASSIFICAZIONE_ISSUE_CUP"] = tmp_df.CODICE_CUP.apply(classify_CUP)

print('Numero di CUP da tmp_df: ',tmp_df.CODICE_CUP.count())
print('Numero di CUP di progetti da PAdigitale2026: ',len(df_pad2026))
# crea aggregati
tmp_df.groupby(by=["CLASSIFICAZIONE_ISSUE_CUP"]).CODICE_CUP.count().to_csv(
    "data/KPI_aggregati.csv", index=True
)

tmp_df.to_excel("data/CUP_ANALYSIS.xlsx", index=False)